In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [10]:
# Dados Entrada

ger = ['Gerador_1', 'Gerador_2', 'Gerador_3']
Ng = len(ger)
pmin = [120,30,80]
pmax = [530,180,250]
c_ener = [100,150,300]

loads = ['Carga_1']
Nc = len(loads)
dem = [780]

#Modelo Concreto:
model = ConcreteModel()

#Sets:
model.ger = Set(initialize=ger)
model.loads = Set(initialize=loads)

# Parâmetros:
model.pmin = Param(model.ger, initialize={ger[k]: pmin[k] for k in range(Ng)})
model.pmax = Param(model.ger, initialize={ger[k]: pmax[k] for k in range(Ng)})
model.c_ener = Param(model.ger, initialize={ger[k]: c_ener[k] for k in range(Ng)})
model.dem = Param(model.loads, initialize={loads[k]: dem[k] for k in range(Nc)})

#Variáveis:
model.prod = Var(model.ger, domain = NonNegativeReals)

In [11]:
#Restrições:
def prod_min(model, gerador):
    return model.prod[gerador] >= model.pmin[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.prod[gerador] <= model.pmax[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def balanco(model):
    return sum(model.prod[i] for i in model.ger) == sum(model.dem[j] for j in model.loads)
model.balanco = Constraint(rule=balanco)

#Função Objetivo:
def objective_rule(model):
    return sum(model.c_ener[i]*model.prod[i] for i in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Executar Otimização:
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

In [12]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.prod[gerador].value))

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 102500.0 

Energia gerada do Gerador_1: 530.00 MWh
Energia gerada do Gerador_2: 170.00 MWh
Energia gerada do Gerador_3: 80.00 MWh
